In [2]:
from qdrant_client import QdrantClient, models

/workspaces/llms/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
client = QdrantClient("http://localhost:6333")

### Study the dataset

In [5]:
import requests 

docs_url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json"
docs_respons = requests.get(docs_url)
documents_raw = docs_respons.json()

### FastEmbed for Textual data

In [6]:
from fastembed import TextEmbedding
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [7]:
import json 

EMBEDDING_DIMENSIONALITY = 512 

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [8]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

### Create a collection

In [ ]:
collection_name = "zoomcamp-rag"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

### Create, Embed & Insert Points into the Collection

In [10]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:
        point = models.PointStruct(
            id = id,
            vector = models.Document(text=doc['text'], model=model_handle), # embed text locally with jinaai from FastEmbed
            payload = {
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']

            } # save ann leeded metadata fields


        )

        points.append(point)

        id += 1

In [12]:
# download the model, and then upsert the generated points into the collection

client.upsert(
    collection_name=collection_name,
    points=points[:100]
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

### Running a similarity Search

In [ ]:
def search(query, limit=3):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( # embed the query text locally with jinaai
            text=query,
            model=model_handle

        ),
        limit=limit,  # top closest matches
        with_payload=True # to get metadata in the results

    )

    return results

In [14]:
import random

course = random.choice(documents_raw)
course_piece = random.choice(course['documents'])
print(json.dumps(course_piece))

{"text": "We can import precision_recall_curve from scikit-learn and plot the graph as follows:\nfrom sklearn.metrics import precision_recall_curve\nprecision, recall, thresholds = precision_recall_curve(y_val, y_predict)\nplt.plot(thresholds, precision[:-1], label='Precision')\nplt.plot(thresholds, recall[:-1], label='Recall')\nplt.legend()\nHrithik Kumar Advani", "section": "4. Evaluation Metrics for Classification", "question": "Quick way to plot Precision-Recall Curve"}


In [15]:
result = search(course_piece['question'])

In [16]:
result

QueryResponse(points=[ScoredPoint(id=39, version=2, score=0.74291325, payload={'text': "The first step is to try to solve the issue on your own. Get used to solving problems and reading documentation. This will be a real life skill you need when employed. [ctrl+f] is your friend, use it! It is a universal shortcut and works in all apps/browsers.\nWhat does the error say? There will often be a description of the error or instructions on what is needed or even how to fix it. I have even seen a link to the solution. Does it reference a specific line of your code?\nRestart app or server/pc.\nGoogle it, use ChatGPT, Bing AI etc.\nIt is going to be rare that you are the first to have the problem, someone out there has posted the fly issue and likely the solution.\nSearch using: <technology> <problem statement>. Example: pgcli error column c.relhasoids does not exist.\nThere are often different solutions for the same problem due to variation in environments.\nCheck the tech’s documentation. U

In [17]:
print(f"Question:\n{course_piece['question']}\n")
print("Top Retrieved Answer:\n{}\n".format(result.points[0].payload['text']))
print("Original Answer:\n{}".format(course_piece['text']))

Question:
Quick way to plot Precision-Recall Curve

Top Retrieved Answer:
The first step is to try to solve the issue on your own. Get used to solving problems and reading documentation. This will be a real life skill you need when employed. [ctrl+f] is your friend, use it! It is a universal shortcut and works in all apps/browsers.
What does the error say? There will often be a description of the error or instructions on what is needed or even how to fix it. I have even seen a link to the solution. Does it reference a specific line of your code?
Restart app or server/pc.
Google it, use ChatGPT, Bing AI etc.
It is going to be rare that you are the first to have the problem, someone out there has posted the fly issue and likely the solution.
Search using: <technology> <problem statement>. Example: pgcli error column c.relhasoids does not exist.
There are often different solutions for the same problem due to variation in environments.
Check the tech’s documentation. Use its search if avai

In [18]:
print(search("What if I submit homeworks late?").points[0].payload['text'])

No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]


### Running Similarity Search with Filters

In [20]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields

)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

In [28]:
def search_in_course(query, course="mlops-zoomcamp", limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must = [
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]

        ),
        limit=limit,
        with_payload=True

    )

    return results

In [32]:
print(search_in_course("What if I submit homeworks late?", "data-engineering-zoomcamp").points[0].payload['text'])

No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]
